# Numerical Integration

In [2]:
%matplotlib inline
import lmfit
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.integrate import odeint

import tellurium as te
import gene_network as gn
import modeling_game as mg
import model_fitting as mf
import gene_analyzer as ga
import util

In [ ]:
def pend(y, t, b, c):
    theta, omega = y
    dydt = [omega, -b*omega - c*np.sin(theta)]
    return dydt

b = 0.25
c = 5.0
y0 = [np.pi - 0.1, 0.0]
t = np.linspace(0, 10, 101)
sol = odeint(pend, y0, t, args=(b, c))

plt.plot(t, sol[:, 0], 'b', label='theta(t)')
plt.plot(t, sol[:, 1], 'g', label='omega(t)')
plt.legend(loc='best')
plt.xlabel('t')
plt.grid()
plt.show()

## lmfit and numerical integration

In [ ]:
df_mrna = mf.cleanColumns(pd.read_csv("wild.csv"))
df_mrna.head()

In [ ]:
# Container for data used in calculations
class Container():
    def __init__(self, df_mrna, desc_stg):
        self.df_mrna = df_mrna
        self._stmt_initializations = util.readFile("model_initializations.txt")
        descriptor = gn.GeneDescriptor.parse(desc_stg)
        self.mrna_name = "mRNA%d" % descriptor.ngene
        self._gene_reaction = gn.GeneReaction(descriptor)
        self._gene_reaction.generate()
        self.ngene = descriptor.ngene
        network = gn.GeneNetwork()
        network.update([desc_stg])
        network.generate()
        self.parameters = network.new_parameters
        self.initializeNamespace()  # self.namespace
        self.mrna_kinetics = self.__class__._transformKinetics(
            self._gene_reaction.mrna_kinetics)
        self.protein_kinetics = self.__class__._transformKinetics(
            "a_protein%d*mRNA%d - d_protein%d*P%d;")
     
    @staticmethod
    def _transformKinetics(kinetics):
        new_kinetics = kinetics.replace("^", "**")
        return new_kinetics.replace(";", "")
    
    def updateParameters(self, parameters):
        self.parameters = parameters
        
    def initializeNamespace(self):
        # Initialize simulatin constants
        self.namespace = {}
        valuesdict = self.parameters.valuesdict()
        for name, value in valuesdict.items():
            self.namespace[name] = value
        exec(self._stmt_initializations, self.namespace)

In [ ]:
container = Container(df_mrna, "7-7A+6")
calcKinetics(np.repeat(0,9), 10.0, container)

In [ ]:
def calcResiduals(parameters):
    """
    Assumes container is in outer scope.
    """
    container.updateParameters(parameters)
    container.initializeNamespace()
    # Construct initial values
    y0_arr = [container.namespace["P%d" % n] for n in range(1, gn.NUM_GENE+1)]
    y0_arr.insert(0, container.namespace[container.mrna_name])
    times = np.array(df_mrna.index.tolist())
    y_mat = odeint(calcKinetics, y0_arr, times, args=(container,))
    residuals = container.df_mrna[container.mrna_name] - y_mat[:, 0]
    return np.var(residuals)

In [ ]:
container = Container(df_mrna, "7+1")
calcResiduals(container.parameters)

In [ ]:
def calcKinetics(y, time, container):
    """
    y[0] = mRNA, y[1],...,y[8] = P1...P8
    # outer scoped: , ngene, df_mrna, kinetics_mrna, kinetics_protein, namespace
    """
    # Adjust time
    time = 10*round(time/10)
    # Update the namespae 
    for idx in range(1, 8):
        protein = "P%d" % idx
        container.namespace[protein] = y[idx]
        col = "mRNA%d" % idx
        container.namespace[col] = container.df_mrna.loc[time, col]
    mRNA = "mRNA%d" % container.ngene
    container.namespace[mRNA] = y[0]
    # Calculate the drivatives
    dydt = [eval(container.mrna_kinetics, container.namespace)]
    for idx in range(1, 9):
        statement = container.protein_kinetics % (idx, idx, idx, idx)
        dydt.append(eval(statement, container.namespace))
    return dydt

In [ ]:
# Do the integration
y0 = np.repeat(0, gn.NUM_GENE + 1)
t = np.array(df_mrna.index.tolist())
#container = Container(df_mrna, "7-7A+6")
sol = odeint(calcKinetics, y0, t, args=(container,))
# NEXT - WRAP A FITTER AROUND THIS

In [ ]:
plt.plot(t, sol)

In [ ]:
plt.plot(t, sol[:,0], t, df_mrna[container.mrna_name])

In [ ]:
plt.plot(t, sol[:,0], t, df_mrna["mRNA7"])

In [ ]:
analyzer = ga.GeneAnalyzer()
analyzer.do("7-7", end_time=100)
analyzer.plot()
